# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [2]:
name = "high_vs_random"
file_name = f"csv/clean_{name}.csv"
df = pd.read_csv(file_name)
df

,Unnamed: 0,text_clean,label
0,0,valueerror too many values to unpack during st...,0
1,1,provide details on how to proceed with setting...,0
2,2,cve201812896 medium detected in linuxlinux46 a...,1
3,3,incorrect recoloring of the background color o...,1
4,4,csw data source australianinstituteofmarinesci...,0
...,...,...,...
158938,158938,support passing of idictionarystring object as...,1
158939,158939,editor overflow menu not shown on api17 expect...,0
158940,158940,client message cant detect virtualbox because ...,1
158941,158941,feature gate story why do we need the feature ...,0


In [3]:
# drop Unnamed: 0
df = df.drop(columns=["Unnamed: 0"])
df

,text_clean,label
0,valueerror too many values to unpack during st...,0
1,provide details on how to proceed with setting...,0
2,cve201812896 medium detected in linuxlinux46 a...,1
3,incorrect recoloring of the background color o...,1
4,csw data source australianinstituteofmarinesci...,0
...,...,...
158938,support passing of idictionarystring object as...,1
158939,editor overflow menu not shown on api17 expect...,0
158940,client message cant detect virtualbox because ...,1
158941,feature gate story why do we need the feature ...,0


In [4]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,79563
0,79380


In [5]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [6]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [7]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 15895
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 127154
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 15894
    })
})

In [8]:
# Uncomment this if not logged in.
#notebook_login()

In [9]:
# Push to Hugging Face Hub
ds.push_to_hub(name+"_min_length_500")

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/128 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/16 [00:00<?, ?ba/s]